# Quelques procédures d'extraction de la base de données
Remarque : certaines données ont été rentrées à la main car les données ont été trop irrégulières et nous n'avons que 23 pays.

In [2]:
from zipfile import ZipFile
import json
liste_dict_pays=[]
with ZipFile('north_america.zip','r') as z:
    for i in z.namelist():
        a=json.loads(z.read(i))
        liste_dict_pays.append(a)

## Extraction du dictionnaire récent pour chaque pays avec wptools 

In [3]:
import wptools

def recup(country):
    page = wptools.page(country,silent=True)
    page.get_parse(False)
    return page.data['infobox']

In [4]:
recup('United_States')

{'conventional_long_name': 'United States of America',
 'common_name': 'the United States',
 'image_flag': 'Flag of the United States.svg',
 'image_coat': 'Greater coat of arms of the United States.svg',
 'symbol_type_article': 'Great Seal of the United States#Obverse',
 'national_motto': '<div style="padding-bottom:0.5em;text-align:center;">"[[In God We Trust]]" </div> {{collapsible list\n|title|=|Other traditional mottos:\n|titlestyle|=|background:transparent;text-align:center;line-height:1.15em;\n|liststyle|=|text-align:center;white-space:nowrap;\n|native phrase|la|"[[E pluribus unum]]"|italics|=|off|ref| name="de facto Motto"|{{cite web|publisher=[[U.S. Department of State]], [[Bureau of Public Affairs]] |year=2003|url= https://2009-2017.state.gov/documents/organization/27807.pdf|title=The Great Seal of the United States|accessdate=February 12, 2020}}|</ref>|rp|6, 15|<br />"Out of many, one"\n|native phrase|la|"[[Annuit cœptis]]"|italics|=|off|ref| name="de facto Motto"|rp|6, 15| <

### Affichage des clés communes

In [5]:
def cle_commune(clé):
    for country in liste_dict_pays:
        if clé not in list(country.keys()):
            return False
    return True

cles_communes=set()
for country in liste_dict_pays:
    for clé in list(country.keys()):
        if cle_commune(clé):
            cles_communes.add(clé)

In [6]:
cles_communes

{'GDP_PPP',
 'GDP_PPP_per_capita',
 'GDP_PPP_year',
 'GDP_nominal',
 'GDP_nominal_per_capita',
 'GDP_nominal_year',
 'HDI',
 'HDI_change',
 'HDI_rank',
 'HDI_year',
 'area_rank',
 'calling_code',
 'capital',
 'cctld',
 'currency',
 'demonym',
 'drives_on',
 'established_date1',
 'established_event1',
 'government_type',
 'image_coat',
 'image_flag',
 'largest_city',
 'leader_name1',
 'leader_name2',
 'leader_title1',
 'leader_title2',
 'legislature',
 'national_anthem',
 'official_languages',
 'population_density_sq_mi',
 'utc_offset'}

## Affichage des exceptions pour une clé :

In [7]:
def exception(clé):
    for country in liste_dict_pays:
        if clé not in list(country.keys()):
            print(country)

In [8]:
exception('conventional_long_name')

{'common_name': 'Grenada', 'native_name': "'''Grenada'''", 'image_flag': 'Flag of Grenada.svg', 'image_coat': 'Coat of arms of Grenada.svg', 'national_motto': '"Ever Conscious of God We Aspire, Build and Advance as One People" {{lower|0.2em|ref|{{cite web |url=http://www.gov.gd |title=Government of Grenada Website |accessdate=2007-11-01}}|</ref>}}', 'national_anthem': "''[[Hail Grenada]]''<br /><center>[[File:HailGrenada.ogg]]</center>", 'royal_anthem': "''[[God Save the Queen]]''", 'image_map': 'Grenada in its region.svg', 'map_caption': 'Map indicating the location of Grenada in the [[Lesser Antilles]]', 'capital': "[[St. George's, Grenada|St. George's]]", 'coordinates': '{{Coord|12|03|N|61|45|W|type:city}}', 'largest_city': 'capital', 'official_languages': '{{unbulleted list |[[English language|English]]}}', 'regional_languages': '<small>[[Grenadian Creole English]]<br /> [[Grenadian Creole French]]</small>', 'ethnic_groups': '{{vunblist\n  | 82% [[Afro-Grenadian|African]]\n  | 13% 

## Récupération du nom officiel à partir du dictionnaire:

In [9]:
def get_name(wp_info):
    
    # cas général
    if 'conventional_long_name' in wp_info:
        name = wp_info['conventional_long_name']        
        return name
    
    #exception :
    if 'common_name' in wp_info:
        name = wp_info['common_name']
        print( 'using common name {}...'.format(name),end='')
        return name

    # Aveu d'échec, on ne doit jamais se retrouver ici
    print('Could not fetch country name {}'.format(wp_info))
    return None

In [10]:
a=recup('Barbados')
get_name(a)

'Barbados'

## Récupération de la capitale :

In [11]:
import re
def get_capital(wp_info):
    
    # cas général
    if 'capital' in wp_info:
        
        # parfois l'information récupérée comporte plusieurs lignes
        # on remplace les retours à la ligne par un espace
        capital = wp_info['capital'].replace('\n',' ')
        
        # le nom de la capitale peut comporter des lettres, des espaces,
        # ou l'un des caractères ',.()|- compris entre crochets [[...]]
        m = re.match(".*?\[\[([\w\s',(.)|-]+)\]\]", capital)
        
        # on récupère le contenu des [[...]]
        capital = m.group(1)
        
        return capital
        
    # Aveu d'échec, on ne doit jamais se retrouver ici
    print(' Could not fetch country capital {}'.format(wp_info))
    return None    

In [12]:
a=recup('United_States')
get_capital(a)

'Washington, D.C.'

## Récupération des coordonnées de chaque capitale

In [12]:
import re

def cv_coords(str_coords):
    # on découpe au niveau des "|" 
    c = str_coords.split('|')

    # on extrait la latitude en tenant compte des divers formats
    lat = float(c.pop(0))
    if (c[0] == 'N'):
        c.pop(0)
    elif ( c[0] == 'S' ):
        lat = -lat
        c.pop(0)
    elif ( len(c) > 1 and c[1] == 'N' ):
        lat += float(c.pop(0))/60
        c.pop(0)
    elif ( len(c) > 1 and c[1] == 'S' ):
        lat += float(c.pop(0))/60
        lat = -lat
        c.pop(0)
    elif ( len(c) > 2 and c[2] == 'N' ):
        lat += float(c.pop(0))/60
        lat += float(c.pop(0))/3600
        c.pop(0)
    elif ( len(c) > 2 and c[2] == 'S' ):
        lat += float(c.pop(0))/60
        lat += float(c.pop(0))/3600
        lat = -lat
        c.pop(0)

    # on fait de même avec la longitude
    lon = float(c.pop(0))
    if (c[0] == 'W'):
        lon = -lon
        c.pop(0)
    elif ( c[0] == 'E' ):
        c.pop(0)
    elif ( len(c) > 1 and c[1] == 'W' ):
        lon += float(c.pop(0))/60
        lon = -lon
        c.pop(0)
    elif ( len(c) > 1 and c[1] == 'E' ):
        lon += float(c.pop(0))/60
        c.pop(0)
    elif ( len(c) > 2 and c[2] == 'W' ):
        lon += float(c.pop(0))/60
        lon += float(c.pop(0))/3600
        lon = -lon
        c.pop(0)
    elif ( len(c) > 2 and c[2] == 'E' ):
        lon += float(c.pop(0))/60
        lon += float(c.pop(0))/3600
        c.pop(0)
    
    # on renvoie un dictionnaire avec les deux valeurs
    return {'lat':lat, 'lon':lon }

def get_coords(wp_info):

    # S'il existe des coordonnées dans l'infobox du pays
    # (cas le plus courant)
    if 'coordinates' in wp_info:

        # (?i) - ignorecase - matche en majuscules ou en minuscules
        # ça commence par "{{coord" et se poursuit avec zéro ou plusieurs
        #   espaces suivis par une barre "|"
        # après ce motif, on mémorise la chaîne la plus longue possible
        #   ne contenant pas de },
        # jusqu'à la première occurence de "}}"
        m = re.match('(?i).*{{coord\s*\|([^}]*)}}', wp_info['coordinates'])

        # l'expression régulière ne colle pas, on affiche la chaîne analysée pour nous aider
        # mais c'est un aveu d'échec, on ne doit jamais se retrouver ici
        if m == None :
            print(' Could not parse coordinates info {}'.format(wp_info['coordinates']))
            return None

        # cf. https://en.wikipedia.org/wiki/Template:Coord#Examples
        # on a récupère une chaîne comme :
        # 57|18|22|N|4|27|32|W|display=title
        # 44.112|N|87.913|W|display=title
        # 44.112|-87.913|display=title
        str_coords = m.group(1)

        # on convertit en numérique et on renvoie
        if str_coords[0:1] in '0123456789':
            return cv_coords(str_coords)
        
    # Aveu d'échec, on ne doit jamais se retrouver ici
    print(' Could not fetch country coordinates')
    return None



In [13]:
e=recup('Canada')
get_coords(e)

{'lat': 45.4, 'lon': -75.66666666666667}

## Récupération de l'HDI :

### Analyse préalable de l'année de HDI :

### Récupération

In [14]:
def get_hdi(wp_info):
    return wp_info['HDI']

In [15]:
a=recup('Panama')
get_hdi(a)

'0.795'

## Récupération des noms de drapeaux :

Une analyse des noms de fichiers permet de voir que les noms de drapeaux commencent par le nom du pays en minuscule avec '_' comme séparateur (remplace l'espace). Exemple : 'united_states...png' pour les Etats-Unis

In [16]:
from zipfile import ZipFile
def get_flag(country):
    with ZipFile('flags.zip','r') as z:
        for i in z.namelist():
            if country in i:
                return i

In [17]:
get_flag('united_states')

'united_states-190x100.png'

In [18]:
import sqlite3

#
# Ouverture d'une connexion avec la base de données
#
conn = sqlite3.connect('pays.sqlite')

In [20]:
recup('United_States')

{'conventional_long_name': 'United States of America',
 'common_name': 'the United States',
 'image_flag': 'Flag of the United States.svg',
 'image_coat': 'Greater coat of arms of the United States.svg',
 'symbol_type_article': 'Great Seal of the United States#Obverse',
 'national_motto': '<div style="padding-bottom:0.5em;text-align:center;">"[[In God We Trust]]" </div> {{collapsible list\n|title|=|Other traditional mottos:\n|titlestyle|=|background:transparent;text-align:center;line-height:1.15em;\n|liststyle|=|text-align:center;white-space:nowrap;\n|native phrase|la|"[[E pluribus unum]]"|italics|=|off|ref| name="de facto Motto"|{{cite web|publisher=[[U.S. Department of State]], [[Bureau of Public Affairs]] |year=2003|url= https://2009-2017.state.gov/documents/organization/27807.pdf|title=The Great Seal of the United States|accessdate=February 12, 2020}}|</ref>|rp|6, 15|<br />"Out of many, one"\n|native phrase|la|"[[Annuit cœptis]]"|italics|=|off|ref| name="de facto Motto"|rp|6, 15| <